In [1]:
from common import load_data, validation

C:\ProgramData\Anaconda3\lib\site-packages\h5py\__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
from models import BaseModel

In [3]:
X_train, Y_train, X_test, Y_test = load_data()

In [4]:
num_train, height, width, depth = X_train.shape
num_classes = Y_train.shape[1]

In [5]:
model = BaseModel(depth, height, width, num_classes)

In [ ]:
validation(model, X_train, Y_train, X_test, Y_test)